# Análise de Campanha de Marketing de Vinhos
Este notebook realiza a limpeza e análise inicial dos dados.

In [ ]:
# Instalar bibliotecas necessárias (se estiver rodando localmente)
# !pip install pandas openpyxl matplotlib seaborn

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações visuais
sns.set(style="whitegrid")


In [ ]:
# Fazer upload do arquivo Excel
from google.colab import files
uploaded = files.upload()

In [ ]:
# Ler o arquivo Excel
xls = pd.ExcelFile(next(iter(uploaded)))
print("Planilhas disponíveis:", xls.sheet_names)

# Carregar a planilha 'data'
df = xls.parse('data')

# Visualizar as primeiras linhas
df.head()

In [ ]:
# Converter a coluna de datas e remover valores ausentes em 'Income'
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], errors='coerce', dayfirst=True)
df = df.dropna(subset=['Income'])

# Verificar se há valores ausentes restantes
df.isnull().sum().sort_values(ascending=False).head()

In [ ]:
# Verificar tipos de dados
df.dtypes

In [ ]:
# Exemplo de visualização: Distribuição de renda
plt.figure(figsize=(10,5))
sns.histplot(df['Income'], bins=30, kde=True)
plt.title('Distribuição da Renda')
plt.xlabel('Renda')
plt.ylabel('Frequência')
plt.show()